In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11438
2,Huelva,Confirmados PDIA 14 días,636
3,Huelva,Tasa PDIA 14 días,"124,52446729568491"
4,Huelva,Confirmados PDIA 7 días,280
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,8
629,Municipio de Huelva sin especificar,Total Confirmados,135
630,Municipio de Huelva sin especificar,Curados,115


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11438.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3252.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1545 personas en los últimos 7 días 

Un positivo PCR cada 704 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11438.0,280.0,636.0,510743.0,54.822093,124.524467,100.0
Huelva-Costa,6483.0,172.0,410.0,288115.0,59.698384,142.304288,52.0
Condado-Campiña,3508.0,74.0,185.0,155057.0,47.724385,119.310963,36.0
Huelva (capital),3252.0,93.0,204.0,143663.0,64.734831,141.998984,27.0
Moguer,405.0,20.0,37.0,22088.0,90.546903,167.511771,15.0
Sierra de Huelva-Andévalo Central,1313.0,26.0,30.0,67571.0,38.478045,44.397745,11.0
Isla Cristina,478.0,23.0,59.0,21264.0,108.164033,277.464259,9.0
Ayamonte,351.0,15.0,22.0,20946.0,71.612718,105.031987,8.0
Palos de la Frontera,267.0,14.0,31.0,11289.0,124.014527,274.603596,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),48.0,9.0,11.0,2364.0,380.710660,465.313029,1.0
Villablanca,68.0,10.0,13.0,2848.0,351.123596,456.460674,1.0
Cumbres Mayores,17.0,8.0,8.0,1755.0,455.840456,455.840456,6.0
Lucena del Puerto,110.0,5.0,14.0,3371.0,148.323939,415.307031,4.0
Palma del Condado (La),561.0,15.0,39.0,10761.0,139.392250,362.419849,6.0
Manzanilla,52.0,7.0,7.0,2135.0,327.868852,327.868852,0.0
Bonares,68.0,2.0,17.0,6058.0,33.014196,280.620667,1.0
Isla Cristina,478.0,23.0,59.0,21264.0,108.164033,277.464259,9.0
Palos de la Frontera,267.0,14.0,31.0,11289.0,124.014527,274.603596,6.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bonares,68.0,2.0,17.0,6058.0,33.014196,280.620667,1.0,0.117647
Rociana del Condado,122.0,1.0,6.0,7855.0,12.730745,76.384468,1.0,0.166667
Cartaya,433.0,9.0,42.0,19974.0,45.058576,210.273355,3.0,0.214286
Aljaraque,445.0,5.0,20.0,21260.0,23.518344,94.073377,2.0,0.250000
Almonte,450.0,3.0,12.0,24191.0,12.401306,49.605225,0.0,0.250000
Lepe,984.0,12.0,37.0,27431.0,43.746127,134.883890,2.0,0.324324
Lucena del Puerto,110.0,5.0,14.0,3371.0,148.323939,415.307031,4.0,0.357143
Palma del Condado (La),561.0,15.0,39.0,10761.0,139.392250,362.419849,6.0,0.384615
Isla Cristina,478.0,23.0,59.0,21264.0,108.164033,277.464259,9.0,0.389831
